# Make API call to KEGG compounds and download all data

### Depends on KEGG API setup

More info: https://www.kegg.jp/kegg/rest/keggapi.html

In [2]:
import requests as req
from io import StringIO
import pandas as pd
import data_utils as du

data_dir = du.find_data_dir('app')

kegg_compounds_csv_file = du.get_file_path(data_dir, 'Kegg compounds', 'csv', 'kegg_compounds.csv')
kegg_compounds_pickle_file = du.get_file_path(data_dir, 'Kegg compounds', 'pickle', 'kegg_compounds.p')

#### Functions

In [74]:
def convert_kegg_compounds_df_to_dicts(kegg_compounds_df):
    kegg_compounds = kegg_compounds_df.to_dict('records')

    for kegg_compound in kegg_compounds:
        kegg_compound['names'] = kegg_compound['name'].split(';')
        del kegg_compound['name']
    
    return kegg_compounds

def download_kegg_compounds():
    operation = 'list'
    argument = 'compound'
    kegg_response = req.get(f"http://rest.kegg.jp/{operation}/{argument}/")

    kegg_compounds_df = pd.read_csv(StringIO(kegg_response.content.decode("utf-8")), delimiter ='\t', names=['accession', 'name'])
    return kegg_compounds_df

#### Download KEGG compounds

In [75]:
kegg_compounds_df = download_kegg_compounds()
kegg_compounds = convert_kegg_compounds_df_to_dicts(kegg_compounds_df)

#### Export to data storage

In [77]:
kegg_compounds_df.to_csv(kegg_compounds_csv_file, index=False)
du.dump_in_pickle(kegg_compounds_pickle_file, kegg_compounds)

#### Import from data storage

In [83]:
kegg_compounds_df = pd.read_csv(kegg_compounds_csv_file)
kegg_compounds = du.read_from_pickle(kegg_compounds_pickle_file)